
```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝
```
**Crea un Servidor de Minecraft en Google Colab! (Versión Modificada en Español)**

---

El Script de abajo iniciará tu servidor. Primero tendrás que crear el servidor para poder usarlo, no te preocupes, los scripts de abajo harán la gran mayoría del trabajo para ti. A lo mejor quieres cambiar la región del servidor, fiajte abajo.

Original: https://github.com/N-aksif-N/MineColab_Improved

----

In [ ]:
# @markdown ##**Instalación** {display-mode: "form"}

# @markdown Mira [la wiki de este proyecto](https://github.com/N-aksif-N/MineColab_Improved/wiki) para más aclaraciones.
from requests import get, post
from time import sleep
from json import load, dump
from os import environ
from os.path import exists
try: from jproperties import Properties
except:
  %pip install -q jproperties
  from jproperties import Properties
try: from rich import print
except:
  %pip install -q rich
  from rich import print
if exists('/content/drive') == False:
  from google.colab import drive
  drive.mount('/content/drive')
from google.colab import files
try: from pyngrok import conf, ngrok
except:
  %pip install -q pyngrok
  from pyngrok import conf, ngrok
try: from bs4 import BeautifulSoup
except:
  %pip install -q BeautifulSoup4
  from bs4 import BeautifulSoup
try: import yaml
except:
  %pip install -q pyyaml
  import yaml
drive_path = '/content/drive/MyDrive/minecraft'
#------------------------------------------------------------------------------------------------------------------------------------#
def LOG(*args, sep=''):
  check = False
  args = list(args)
  for i in range(len(args)):
    args[i] = str(args[i])
    if '\n' in args[i]: args[i] = args[i].replace('\n', ''); args.insert(0, '\n[bold green][ LOG ][/bold green] '); check = True; break
  if check == False: args.insert(0, '[bold green][ LOG ][/bold green] ')
  print(sep.join(map(str, args)))
def GET(url):
  r = get(url)
  if r.status_code == 200:
    return r
  else: raise Exception('Error '+ str(r.status_code) + "! Most likely you entered an unsupported version. Try running the code again if you think that shouldn't have happened.")
def DOWNLOAD_FILE(url, path, file_name):
  # Download file into file_name thourgh url
  r = GET(url)
  LOG('\nDownloading ' + file_name)
  with open(path + '/' + file_name, 'wb') as f:
    f.write(r.content)

def COLABCONFIG(server_name):
  return f"{drive_path}/{server_name}/colabconfig.txt"
def COLABCONFIG_LOAD(server_name):
  if exists(COLABCONFIG(server_name)): return load(open(COLABCONFIG(server_name)))
  else:
    if exists(COLABCONFIG(server_name)): dump({"server_type": "generic"}, open(COLABCONFIG(server_name), 'w')) # using default, if config doesn't exist.
    raise Exception('Please checking whether you deleted your colabconfig file or not.')

SERVERCONFIG = f'{drive_path}/server_list.txt'
def SERVER_IN_USE(server_name):
  if exists(f'{drive_path}/{server_name}') and server_name != '': return server_name
  else:
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_in_use'] != '': return serverconfig['server_in_use']
    else:
      raise Exception('Please create a minecraft server first!')
# The jar file name
def JAR_LIST_RUN(server_version):
  return {'generic': 'server.jar', 'vanilla':'server.jar','snapshot': 'server.jar',
          'purpur' : 'server.jar', 'paper': 'server.jar', 'velocity' : 'server.jar',
          'fabric' : 'server.jar',
          'forge': 'minecraft_server.' + server_version + '.jar',
          'arclight' : 'server.jar', 'mohist': 'server.jar', 'banner': 'server.jar'}
#------------------------------------------------------- INSTALL SERVER.JAR -------------------------------------------------------------#
class Server:
  def __init__(self, server_name, server_type= '', version= ''):
    self.server_name = server_name
    self.server_type = server_type
    self.version = version
  def SERVERSJAR(self, all = False, jar = False):
    #Get the download URL (jar) AND return the detailed versions for each software (all)
    server_type = self.server_type
    version= self.version
    if all:
      Server_Jars_All = {
        'paper': 'https://api.papermc.io/v2/projects/paper', 'velocity': 'https://api.papermc.io/v2/projects/velocity',
        'purpur': 'https://api.purpurmc.org/v2/purpur',
        'mohist': 'https://mohistmc.com/api/v2/projects/mohist', 'banner': 'https://mohistmc.com/api/v2/projects/banner'
      }
      if server_type == 'vanilla' or server_type=='snapshot':
        rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
        if server_type == 'vanilla': server_type = 'release'
        if version != 'vanilla - latest_version': server_version = [hit["id"] for hit in rJSON["versions"] if hit["type"] == server_type]
        else:
          return rJSON['latest']['release']
      elif server_type == 'paper' or  server_type == 'velocity' or server_type == 'purpur' or server_type == 'mohist' or server_type == 'banner':
        rJSON = GET(Server_Jars_All[server_type]).json()
        server_version = [hit for hit in rJSON["versions"]]
      elif server_type == 'fabric':
        rJSON = GET('https://meta.fabricmc.net/v2/versions/game').json()
        server_version = [hit['version'] for hit in rJSON if hit['stable'] == True]
      elif server_type == 'forge':
        rJSON = GET('https://files.minecraftforge.net/net/minecraftforge/forge/index.html')
        soup = BeautifulSoup(rJSON.content, "html.parser")
        server_version = [tag.text for tag in soup.find_all('a') if '.' in tag.text and '\n' not in tag.text]
      else:
        LOG('Before going deeper, please check out https://github.com/IzzelAliz/Arclight')
        rJSON = GET('https://files.hypoglycemia.icu/v1/files/arclight/minecraft').json()['files']
        server_version  = [hit['name'] for hit in rJSON]
      return server_version
    elif jar == True and version != None:
      # RETURN DOWNLOAD URL
      if server_type == 'vanilla' or server_type=='snapshot':
        rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
        if server_type == 'vanilla': server_type = 'release'
        for hit in rJSON["versions"]:
          if hit["type"] == server_type and hit['id'] == version:
            return GET(hit['url']).json()["downloads"]['server']['url']
      elif server_type == 'paper' or  server_type == 'velocity':
        build = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}').json()["builds"][-1]
        jar_name = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}').json()["downloads"]["application"]["name"]
        return f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}/downloads/{jar_name}'
      elif server_type == 'purpur':
        build = GET(f'https://api.purpurmc.org/v2/purpur/{version}').json()["builds"]["latest"]
        return f'https://api.purpurmc.org/v2/purpur/{version}/{build}/download'
      elif server_type == 'mohist' or server_type == 'banner':
        return GET(f'https://mohistmc.com/api/v2/projects/{server_type}/{version}/builds').json()["builds"][-1]["url"]
      elif server_type == 'fabric':
        installerVersion = GET('https://meta.fabricmc.net/v2/versions/installer').json()[0]["version"]
        fabricVersion = GET(f'https://meta.fabricmc.net/v2/versions/loader/{version}').json()[0]["loader"]["version"]
        return "https://meta.fabricmc.net/v2/versions/loader/" + version + "/" + fabricVersion + "/" + installerVersion + "/server/jar"
      elif server_type == 'forge':
        rJSON = GET(f'https://files.minecraftforge.net/net/minecraftforge/forge/index_{version}.html')
        soup = BeautifulSoup(rJSON.content, "html.parser")
        tag =  soup.find('a', title="Installer"); tag = str(tag); tag = tag[tag.find('"') + 1 :]
        link = tag[:tag.find('"')]; link = link[link.find('=') + 1:]; link = link[link.find('=') + 1:]
        return link
      else:
        rJSON = GET(f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders').json()
        LOG('Available type: '); print([hit['name'] for hit in rJSON['files']])
        build = input(' Type: '); choice = input('Stable(st) or Snapshot(sn): ')
        if choice == 'sn': choice = 'latest-snapshot';
        else: choice = "latest-stable";
        return f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders/{build}/{choice}'
    else: raise Exception('Wrong given arguments')
  def Install_server(self, tunnel_service):
    # Auditing whether file is existed.
    if exists(f'{drive_path}/{self.server_name}') == False:
      # Create folder
      ! mkdir -p $drive_path/$self.server_name
      # Get version
      if self.version == '':
        # Print available version => get version (string variable)
        LOG(f'Latest versions/builds for {self.server_type}')
        server_version = self.SERVERSJAR(all = True)
        print('\n Aivailable version/builds:\n', server_version)
        version = input('Server versions: ')
        while version == '':
          LOG("Invalid versions")
          print('\n Aivailable version/builds:\n', server_version)
          version = input('Server version: ')
        self.version = version
      elif self.version == 'vanilla - latest_version': self.version = self.SERVERSJAR(all = True)
      # Load serverconfig
      serverconfig = load(open(SERVERCONFIG))
      serverconfig['server_list'] += [self.server_name]
      serverconfig['server_in_use'] = self.server_name
      if serverconfig['ngrok_proxy'] == {"authtoken": "", "region": ""} and tunnel_service == 'ngrok':
        LOG('Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
        serverconfig['ngrok_proxy']['authtoken'] = input('Your authtoken: ')
        LOG('Available Regions:', ' ap - Asia/Pacific (Singapore)', ' au - Australia (Sydney)', ' eu - Europa (Frankfurt - Germany)', ' in - India (Mumbai)', ' jp - Japan (Tokyo)', ' sa - America (São Paulo - Brazil)', ' us - United States (Ohio)', sep='\n')
        serverconfig['ngrok_proxy']['region'] = input('Region: ')
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      # Set up colabconfig
      colabconfig = {"server_type": self.server_type, "server_version": self.version, "tunnel_service" : tunnel_service}
      dump(colabconfig, open(COLABCONFIG(self.server_name),'w'))
      # Download jar file
      if self.server_type == 'forge': jarname = 'forge-installer.jar' # The jar file name (forge need a special process)
      else: jarname = JAR_LIST_RUN(self.version)[self.server_type]
      # Get jar file
      DOWNLOAD_FILE(url= self.SERVERSJAR(jar = True), path = f"{drive_path}/{self.server_name}", file_name= jarname)
      LOG('\nCompleted!')
    else: raise Exception('Lol, you have already installed this server file')
  def Delete_server(self):
    # Get default server
    server_name = SERVER_IN_USE(self.server_name)
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_list'] == []: raise Exception("You haven't installed yet.")
    else:
      # Auditing whether file is existed.
      if exists(f'{drive_path}/{server_name}') == False: raise Exception("You haven't installed yet.")
      LOG(f'Deleting {server_name}...')
      # Delete folder without noticable
      !rm -rf "{drive_path}/{server_name}" &>/dev/null&
      # Remove the folder name in server config txt files
      serverconfig['server_list'].remove(server_name)
      if serverconfig['server_in_use'] == server_name:
        try: serverconfig['server_in_use']= serverconfig['server_list'][0]
        except: serverconfig['server_in_use'] = ''
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      LOG('Completed')
  @staticmethod
  def Choose_server():
    # Get default server
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_list'] == []: raise Exception(' Creating your minecraft server before choosing')
    else:
      LOG('\nAivailable server:\n')
      for server in serverconfig['server_list']:
        LOG(server)
      serverconfig['server_in_use'] = input('\nYour minecraft server: ')
      dump(serverconfig, open(SERVERCONFIG, 'w'))
      LOG('Completed')

class Cmd:
  def __init__(self, server_name):
    self.server_name = SERVER_IN_USE(server_name)
    self.colabconfig = COLABCONFIG_LOAD(self.server_name)
    self.serverconfig = load(open(SERVERCONFIG))
    self._type = self.colabconfig["server_type"]
    self.version = self.colabconfig["server_version"]
    self.tunnel_service = self.colabconfig["tunnel_service"]
  def INSTALL_JAVA(self):
    # Adoptopen jdk -- optional
    # !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
    # !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk gets installed."

    # Java 8 is required to run Minecraft versions 1.12 through 1.17. Java 17 is required to run Minecraft version 1.18 and up.
    if self.version >= '1.20.5':
      ! sudo apt-get install -y openjdk-21-jre-headless  > /dev/null && echo 'Openjdk21 is working correctly, you are good to go.' || "Openjdk21 doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.20.5 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64/bin/java"
      ! update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
    elif self.version >= "1.17":
      ! sudo apt-get -y install openjdk-17-jre-headless  > /dev/null && echo 'Openjdk17 is working correctly, you are good to go.' || "Openjdk17 doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
      ! update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
    else:
      ! sudo apt-get -y install openjdk-8-jre-headless   > /dev/null && echo 'Openjdk8 is working correctly, you are good to go.' || echo "Openjdk8 doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
      ! update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  def CONNECT_NGROK(self, port, type_ , proxy):
    # Get serverconfig['ngrok_proxy'] : dict includes (authtoken, region)
    token = proxy['authtoken']
    !ngrok authtoken $token
    region = proxy['region']
    conf.get_default().region = region
    url = ngrok.connect (port, type_)
    if type_ == 'tcp': return 'Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', '')
    else: return url
  def EXIT_NGROK(self):
    # Exiting ngrok tunnel helps improve the performance
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels: ngrok.disconnect(tunnel.public_url)
    ngrok.kill()
    LOG("Ngrok closed.")
  def RUNCOMMAND(self, hide = False):
    # Get all the improving java arguments
    java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
    args = " -Xms8G -Xmx8G"
    if self._type == "paper" or self._type == 'purpur':
      # Improving paper cilent (purpur is an alternative).
      # For more detailed: https://docs.papermc.io/paper/aikars-flags
      args += ' -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true'
    elif self._type == "velocity":
      # Java argument for velocity server
      # For more detailed: https://docs.papermc.io/velocity/tuning
      args += ' -XX:+UseG1GC -XX:G1HeapRegionSize=4M -XX:+UnlockExperimentalVMOptions -XX:+ParallelRefProcEnabled -XX:+AlwaysPreTouch -XX:MaxInlineLevel=15'
    # GC_LOGGING
    if java_ver[0] == "1": args += ' -Xloggc:gc.log -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps -XX:+UseGCLogFileRotation -XX:NumberOfGCLogFiles=5 -XX:GCLogFileSize=1M'
    # else: args += '-Xlog:gc*:logs/gc.log:time,uptime:filecount=5,filesize=1M' # GC_Logging may not worked in java 11 and above

    # Forge need to open and run it before starting server. => Using command instead.
    if self._type == 'forge':
      !java -jar forge-installer.jar --installServer >/dev/null
    jar_name = JAR_LIST_RUN(self.version)[self._type]
    # Set up run server cmd
    cmd = ''
    try:
      with open(f'{drive_path}/{self.server_name}/run.sh', 'r') as f:
        cmd = f.read()
    except: pass
    if cmd != '':
      cmd = cmd[cmd.find('java'): ].replace('@user_jvm_args.txt', args)
      cmd = cmd.replace('"$@"', 'nogui "$@"')
    else: cmd = f'java -server {args} -jar {jar_name} nogui'

    if hide == True:
      if self._type == 'forge':
        # Make run.sh running
        if exists(f'{drive_path}/{self.server_name}/run.sh') == False:
          LOG('Run the server aigain to finished installing.')
      if self._type != 'mohist' and self._type != 'banner':
        # Install needed file: eula.txt and so on.
        ! $cmd >/dev/null && wait&
    else:
      # Starting tunneling and run java file.
      LOG(f'Starting server ({self.tunnel_service})...')
      LOG('Stop server => /stop')
      if self.tunnel_service == "ngrok":
        LOG(self.CONNECT_NGROK(type_= 'tcp', port= 25565, proxy = self.serverconfig['ngrok_proxy']))
        ! $cmd
      elif self.tunnel_service == "playit":
        # Download playit
        ! command -v playit || curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg &>/dev/null && echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list && sudo apt -qq update && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
        # Run server
        LOG('Playit will run automatically in the background. To stop the playit service, run sudo systemctl stop playit ')
        ! playit setup
        ! playit &> /dev/null & $cmd
      else:
        # Download argo
        if exists(f'{drive_path}/{self.server_name}/cloudflared-linux-amd64') == False:
          ! curl https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 > cloudflared-linux-amd64
          ! chmod +x cloudflared-linux-amd64
        # Run server
        !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 && echo 'Your free tunnel has started!' && $cmd
  def Geyser_config(self):
    if 'Geysermc' in self.colabconfig:
      if 'fabric' in self._type: path = f'{drive_path}/{self.server_name}/config/Geyser-Fabric/config.yml'
      elif'paper' in self._type or 'purpur' in self._type: path = f'{drive_path}/{self.server_name}/plugins/Geyser-Spigot/config.yml'
      else:
        LOG('Check out https://wiki.geysermc.org/floodgate/setup/ for installing Floodgate on servers behind the proxy')
        LOG('Note for Fabric behind a Velocity proxy: You will need to configure FabricProxyLite to allow the Fabric server to receive data from Velocity.')
        LOG('After done settings, copy the key.pem file in the proxy Floodgate config folder to all backend servers’ Floodgate config folder.')
        LOG('DO NOT DISTRIBUTE THIS KEY TO ANYBODY! This key is what allows for Bedrock accounts to bypass the Java Edition authentication, and if anyone gets ahold of this, they can wreak havoc on your server.')
        path =  f'{drive_path}/{self.server_name}/plugins/Geyser-Velocity/config.yml'
      # To get the requirement
      if exists(path):
        LOG('Config geyser plugin')
        # Configuring
        with open(path, 'r') as file:
            config = yaml.safe_load(file)
            config['bedrock']['clone-remote-port'] = 'true'
            config['auth-type'] = 'floodgate'
            yaml.dump(config, file)
        # Clear notification
        self.colabconfig.pop('Geysermc')
        dump(self.colabconfig, open(COLABCONFIG(self.server_name),'w'))
        LOG('Config done. Check out https://wiki.geysermc.org/geyser/playit-gg for setting tunnels')
        LOG('Verify whether connections from other networks are possible by running geyser connectiontest <ip>:<port> in the console.')
  def Run_server(self):
    # Cd to server path
    %cd $drive_path/$self.server_name
    # Install java acording to the version of server.jar
    self.INSTALL_JAVA()
    # Install necessary files
    if exists(f'{drive_path}/{self.server_name}/eula.txt') == False:
      LOG("\n[Note: Downloading essentials files takes approximately 10-15 minutes. Please wait for a little bit.]")
      self.RUNCOMMAND(hide = True)
      # Accept EULA
      LOG('\nAccepting Eula.')
      !sed -i 's/eula=false/eula=true/g' eula.txt
    # See what changes in the folder
    LOG("\nFiles:")
    !ls -a
    # Run minecraft server
    self.RUNCOMMAND()
    self.Geyser_config()
    # Stop minecraft server => Exit tunnel => Cd to home
    LOG("Finalized server.")
    if self.colabconfig['tunnel_service'] == 'ngrok': self.EXIT_NGROK()
    %cd $drive_path

class Download_:
  def __init__(self, choice, server_name, categories, versions, proj_types, index):
    self.server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{self.server_name}/server.properties") == False: raise Exception(' Running your minecraft server before editing properties')
    else:
      self.colabconfig = COLABCONFIG_LOAD(self.server_name)
      if versions == 'default': versions = self.colabconfig["server_version"]
      if categories == 'default': categories = self.colabconfig['server_type']
      if proj_types == 'default':
        if 'fabric' in categories or 'forge' in categories : proj_types = 'mods'
        elif 'paper' in categories or 'purpur' in categories : proj_types = 'plugins'
        elif 'arclight' in categories or 'mohist'  in categories or 'banner': proj_types = input('Project type (mods or plugins or modpacks) : ');
      else: proj_types = 'tmp';
      if 'arclight' in categories or 'mohist'  in categories or 'banner' in categories or 'vanilla' in categories or 'snapshot' in categories: categories = 'none';
      elif 'paper' in categories or 'purpur' in categories:
        if proj_types == 'mods': raise Exception(f'Hmm, maybe you install the wrong types. Your server_type (currently) is {categories}');
      elif 'fabric' in categories or 'forge' in categories:
        if proj_types == 'plugins': raise Exception(f'Hmm, maybe you install the wrong types. Your server_type (currently) is {categories}');
      self.categories = categories
      self.versions = versions
      self.proj_types = proj_types
      self.index = index
      self.path = f'{drive_path}/{self.server_name}/{self.proj_types}'
      self.choice = choice
  def FACETS(self, software):
    # Get all the syntax
    if software == 'Modrinth':
      facets = "["
      if self.categories != 'none': facets += '["categories:' + self.categories + '"]';
      if facets != '[' and self.versions != 'none': facets += "," + '["versions:' + self.versions +'"]'
      elif self.versions != 'none': facets += '["versions:' + self.versions + '"]';
      if self.proj_types == 'mods': proj_types = 'mod'
      elif self.proj_types == 'plugins': proj_types = 'plugin'
      if facets != '[' and proj_types != 'tmp': facets += "," + '["project_type:' + proj_types + '"]'
      elif self.proj_types != 'tmp': facets += '["project_type:' + proj_types + '"]'
      facets += "]"; facetsInURL = "";
      if facets != "[]": facetsInURL += f'&facets={facets}'
      if self.index != "none": facetsInURL += f'&index={self.index}'
      return facetsInURL
    else:
      if self.categories != 'none':
        if self.categories == 'fabric': categories = 4 #Fabric
        elif self.categories == 'forge': categories = 1 #Forge
        elif self.categories == 'quilt': categories = 5 # quilt
        categories = f"&modLoaderType={categories}"
      if self.index != "none":
        if self.index == "relevance": index = 1 # Featured
        elif self.index == "downloads": index = 6 #TotalDownloads
        elif self.index == "follows": index = 2 #Popularity
        elif self.index == "newest": index = 11 #ReleasedDate
        elif self.index == "updated": index = 3 #LastUpdated
        index = f"&sortField={index}"
      else: index = ''
      if self.versions != "none": versions = f"&gameVersion={versions}"
      else: versions = ""
      return categories + versions + index
  def SEARCH(self, search_name, software):
    project = {}
    LOG(f'\nSearching for the related of {search_name} ...\n')
    facetsInURL = self.FACETS(software)
    if self.software == "Modrinth":
      # Get syntax and get data
      rJSON = GET(f'https://api.modrinth.com/v2/search?query={search_name}{facetsInURL}').json()
      # Get the list of all relevant project
      for hit in rJSON['hits']:
        # Auditing if it for server or not.
        if hit['server_side'] == 'optional' or hit['server_side'] == 'required':
          # Get a full list title : description
          print(hit['slug'], " : ", hit['description'])
          project[hit['slug']] = hit['project_id']
    elif software == 'Curseforge':
      LOG(f"Curseforge doesn't have the exact searching key for cilent-side or server-side => you may get errors when running this {self.proj_types}")
      # Because I haven't found any corresponded of project types in the search engine of Curseforge, I don't use it for searching.
      # The gameid of Minecraft is 432
      rJSON = GET(f"https://api.curse.tools/v1/cf/mods/search?gameId=432&searchFilter={search_name}{facetsInURL}").json()
      # Get the list of all relevant project
      for hit in rJSON["data"]:
        # Get a full list name: summary
        print(hit["name"], ' : ', hit["summary"])
        project[hit['name']] = str(hit["id"])
    # Checking whether your search name wrong or not. If yes => Get the name of project => Get project id
    project_names =''
    if project == {}:
      raise Exception(f"\nSomething went wrong. Please check your search name.")
    else:
      LOG('\nType the project_name you want to download')
      project_names= input('Project_name: ')
      while project_names not in project:
        LOG('\nWrong project_names please type aigain. If you want to quit, type "None".')
        project_names= input('\nProject_name: ')
        if project_names == 'None': raise Exception('Stopping...')
    return [project, project_names]
  def MODPACK(self, file_name, software):
    # unzip modpack
    !unzip file_name -d file_name.split('.')[0] &>/dev/null&
    # Copy the directory in orverrides and paste it (overrides includes the config files of the developer)
    !cp -r $file_name/overrides '{drive_path}/{self.server_name}'
    # Each pahe give th difference file json. The file json give full details
    # (Modrinth: modrinth.index.json- Download link, Curseforge: manifest.json - fileID, projectID) for mods which is included in modpack.
    if software == 'Modrinth':
      manifest = load(open(f"{self.path}/modrinth.index.json"))["files"]
      for files in manifest:
        path_ = files["path"].split("/")[0]
        file_name_ = files["path"].split("/")[1]
        DOWNLOAD_FILE(url = files["downloads"], path = f'{drive_path}/{self.server_name}/{path_}',file_name = file_name_)
    else:
      manifest = load(open(f"{self.path}/manifest.json"))["files"]
      for files in manifest:
        project_id = files["projectID"]
        file_ID = files["fileID"]
        rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}/files/{fileId}').json()["data"]
        file_name_ = rJSON["displayName"]
        DOWNLOAD_FILE(url = files["downloads"], path = f'{drive_path}/{self.server_name}/mods', file_name = file_name_)
  def Install_(self, search_name, software):
    LOG(f'Acessing: {self.server_name}')
    if self.choice == 'url':
      # Find file_name in download url
      filename = input('File name (optional) : ')
      if filename == '':
        filename = url[url.find("/") + 1:]
        while filename.find("/")!= -1: filename = filename[filename.find("/") + 1:]
      # Download file
      DOWNLOAD_FILE(url= self.url, path = self.path, file_name= file_name)
      if self.proj_types == 'modpack': self.MODPACK(file_name= file_name, software= software)
      LOG('\nCompleted')
    elif self.choice == 'upload_file':
      uploaded = files.upload()
      filename = input('File name (optinal)') # The file name which you want to be
      files = [fn for fn in uploaded.keys()][0] # Get the name of the uploaded file
      if filename != '':
        try:
          !cp $drive_path/$files '{self.path}/{filename}.{files[files.find('.'):]}'
        except: raise Exception("Lol, you didn't upload file yet.")
      else:
        !cp $drive_path/$files '{self.path}/{files}'
      !rm -rf $drive_path/$files &>/dev/null&
      if self.proj_types == 'modpack': self.MODPACK(file_name= file_name, software= software)
      LOG('\nCompleted.')
    elif self.choice == 'search':
      a = self.SEARCH(search_name, software)
      project = a[0]; project_names = a[1]; check = False;
      project_id = project[project_names]
      if exists(self.path) == False:
        !mkdir $self.path
      if software == 'Modrinth': rJSON = GET(f'https://api.modrinth.com/v2/project/{project_id}/version').json()
      else: rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}').json()['data']['latestFilesIndexes'];
      for data in rJSON:
        if software == 'Curseforge': gameversions= data["gameVersion"]
        else: gameversions= data["game_versions"]
        if self.versions in gameversions:
          if software == 'Curseforge': files = data, fileId = data['fileId']; url = f'https://www.curseforge.com/api/v1/mods/{project_id}/files/{fileId}/download'
          else: files = data['files'][0]; url = files['url']
          DOWNLOAD_FILE(url= self.url, path = self.path, file_name= files['filename'])
          check = True
        if check == True and self.proj_types == 'modpack':
          self.MODPACK(file_name= file_name, software= software); LOG('\nCompleted.'); break
        elif check: LOG('\nCompleted.'); break
      if check == False: raise Exception(f"It seems that {self.software} doesn't support this {self.proj_types}")
    else:
      if self.categories != 'none' and self.colabconfig['tunnel_service'] == 'playit' and self.categories != 'forge':
        if 'velocity' in self.categories:
          if exists(f'{drive_path}/{server_name}/plugins') == False:
            !mkdir '{drive_path}/{server_name}/plugins'
          plugin = ['ViaVersion', 'ViaBackwards']
          for plugins in plugin:
            rJSON = GET(f'https://hangar.papermc.io/api/v1/projects/{plugins}/versions').json()['result']; check = False
            if self.categories.upper() in rJSON[0]['platformDependenciesFormatted']:
              for hit in rJSON:
                if self.versions in hit['platformDependencies'][self.categories.upper()]:
                  DOWNLOAD_FILE(url = hit["downloads"][self.categories.upper()]["downloadUrl"], path = f'{drive_path}/{self.server_name}/plugins', file_name = hit['downloads'][self.categories.upper()]["fileInfo"]['name'])
                  check = True; break
              if check == False: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
              else: LOG(f'Installing {plugins} done. Try to install  for more supporter version')
            else: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/geyser/versions/latest/builds/latest/downloads/velocity', path = f'{drive_path}/{server_name}/plugins', file_name = 'Geyser-Velocity.jar')
          LOG('You only need to install Floodgate on the BungeeCord or Velocity proxy server, unless you want to use the Floodgate API on the backend servers. Additionally, it will display Bedrock edition skins properly.')
          LOG('For more details. Check out https://wiki.geysermc.org/floodgate/setup/')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/floodgate/versions/latest/builds/latest/downloads/velocity', path = f'{drive_path}/{server_name}/plugins', file_name = 'floodgate-velocity.jar')
        elif 'fabric' in self.categories:
          # warningserver_version
          if self.versions < '1.20.5' and self.versions >= '1.17':
            LOG('Geyser-Fabric run only on 1.20.5/1.20.6.')
            LOG('To use Geyser with an older server version, you can use Geyser on a BungeeCord/Velocity proxy or Geyser-paper instead')
          else: raise Exception(f"Your server_type isn't compatible for running Geyser_MC")
          LOG('Geyser only works with server-side mods. Mods that require a client-side install will not work!')
          LOG('Download geyser mc mods')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/geyser/versions/latest/builds/latest/downloads/fabric', path = f'{drive_path}/{self.server_name}/mods', file_name= 'Geyser-Fabric.jar')
          # Geyser mc mod require fabric api.
          LOG('Download Fabric api (requirement)')
          rJSON = GET(f'https://api.modrinth.com/v2/project/P7dR8mSH/version').json()
          check = False
          for data in rJSON:
            if self.versions in data["game_versions"]:
              files = data['files'][0]; url = files['url']
              DOWNLOAD_FILE(url= self.url, path = self.path, file_name= files['filename']); check = True
            if check: break
          if check == False: raise Exception(f"It seems that theresn't fabric api for this minecraft version")
          LOG('Download floodgate')
          rJSON = GET(f'https://api.modrinth.com/v2/project/bWrNNfkb/version').json()
          check = False
          for data in rJSON:
            if versions in data["game_versions"]:
              files = data['files'][0]; url = files['url']
              DOWNLOAD_FILE(url= self.url, path = self.path, file_name= files['filename']); check = True
            if check: break
          if check == False: raise Exception(f"It seems that theresn't fabric api for this minecraft version")
        elif 'paper' in self.categories or 'purpur' in self.categories:
          if self.versions < '1.20.5' and self.versions >= '1.17':
            plugin = ['ViaVersion', 'ViaBackwards']
            for plugins in plugin:
              rJSON = GET(f'https://hangar.papermc.io/api/v1/projects/{plugins}/versions').json()['result']; check = False
              if self.categories.upper() in rJSON[0]['platformDependenciesFormatted']:
                for hit in rJSON:
                  if self.versions in hit['platformDependencies'][self.categories.upper()]:
                    DOWNLOAD_FILE(url = hit["downloads"][self.categories.upper()]["downloadUrl"], path = f'{drive_path}/{self.server_name}/plugins', file_name = hit['downloads'][self.categories.upper()]["fileInfo"]['name'])
                    check = True; break
                if check == False: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
                else: LOG(f'Installing {plugins} done. Try to install  for more supporter version')
              else: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
          else: raise Exception(f"Your server_type isn't compatible for running Geyser_MC")
          LOG('Download geyser mc plugin')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/geyser/versions/latest/builds/latest/downloads/spigot', path = f'{drive_path}/{self.server_name}/plugins', file_name= 'Geyser-Spigot.jar')
          LOG('Download floodgate')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/floodgate/versions/latest/builds/latest/downloads/spigot', path = f'{drive_path}/{self.server_name}/plugins', file_name = 'floodgate-spigot.jar')
        # Set up notification
        self.colabconfig['Geysermc']= 'notdone'
        dump(self.colabconfig, open(COLABCONFIG(self.server_name),'w'))
        # Ping
        LOG('\nInstalling done. Try to rerun the server to configuration.')
      else: raise Exception(f"Your server_type isn't compatible for running Geyser_MC")
#--------------------------------------------------- Ulities ----------------------------------------------------------#
class Ulities:
  @staticmethod
  def Custom_props(server_name, choice):
    # Default server
    server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Running your minecraft server before editing properties')
    else:
      LOG('Server:  ' + server_name)
      # Add server icon file
      if choice == 'server_icon':
        # Upload image and get direct link from https://postimages.org/
        # or leave it emty. it will download the file.
        url = input('Download_url (able to leave it emty)')
        if url != '': DOWNLOAD_FILE( url = server_icon, path = f'{drive_path}/{server_name}', file_name = 'server_icon.png')
        else:
          uploaded = files.upload()
          files = uploaded.keys()
          try:
            files = [fn for fn in uploaded.keys()][0]
            !cp $drive_path/$files $drive_path/$server_name/'server_icon.png'
          except: raise Exception("You didn't download server_icon.png")
          !rm -rf $drive_path/$files
      else:
        LOG('Get MOTD from https://mctools.org/motd-creator')
        server_properties = Properties()
        with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
            server_properties.load(f, "utf-8")
        server_properties["motd"] = motd
        with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
            server_properties.store(f, encoding="utf-8")
      LOG('Completed')
  @staticmethod
  def Serverproperties(
      server_name, Slots, Gamemode, Difficulty, Cracked, PVP, Command_block, Fly,
      Animals, Monsters, Villagers, Nether, Force_gamemode, Spawn_protection):
    server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Running your minecraft server before editing properties')
    else:
      server_properties = Properties()# Download file
      with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
          server_properties.load(f, "utf-8")
      # Configuring
      server_properties["max-players"] = str(Slots)
      server_properties["gamemode"] = Gamemode
      server_properties["difficulty"] = Difficulty
      dict_ = {'pvp': PVP, 'enable-command-block': Command_block, 'allow-flight': Fly, 'spawn-animals': Animals, 'spawn-monsters': Monsters,
              'spawn-npcs': Villagers, 'allow-nether': Nether, 'force-gamemode': Force_gamemode, 'spawn-protection': Spawn_protection, "online-mode" : Cracked}
      for keys, value in dict.items(): server_properties[keys] = str(value).lower()
      # Saving
      with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
          server_properties.store(f, encoding="utf-8")
      LOG('Completed')
  @staticmethod
  def Software(server_name, server_type, version = ''):
    serverconfig = load(open(SERVERCONFIG))
    if server_name == '': server_name = serverconfig['server_in_use']
    # Delete server
    colabconfig = load(open(COLABCONFIG(server_name)))
    Server(server_name = server_name).Delete_server()
    sleep(25)
    # Install server
    Server(server_name = server_name, server_type = server_type, version = version).Install_server(tunnel_service = colabconfig['tunnel_service'])
  def Logs(server_name):
    server_name = SERVER_IN_USE(server_name)
    # Check log
    if exists(f'/content/drive/My Drive/minecraft/{server_name}/logs/latest.log'):
      LOG(f'Server {server_name}')
      # Open and get log content
      with open(f'/content/drive/My Drive/minecraft/{server_name}/logs/latest.log', 'r') as f:
        content = f.read()
        log = {'content' : content}
        rJSON = post(f'https://api.mclo.gs/1/log/', data = log).json()
        # Fix log. If there's an error
        if rJSON['success'] == 'true':
          id = rJSON['id']; rJSON['raw']
          LOG(f"Raw log file: {raw}")
          rJSON = GET(f'https://api.mclo.gs/1/insights/{id}').json()['analysis']
          problems = rJSON['problems']['message']; solutions = rJSON['solutions']['message']
          LOG('Analysis: \n')
          LOG(f'    Problem: {problems}')
          LOG(f'    Solutions: {solutions}')
        # When log content is too big
        else:
          print(content)
          LOG('\nTry access to https://mclo.gs to see problems and the way to deal with.')
    else: raise Exception("Error: File didn't exists")
  @staticmethod
  def Backup_files(file_name = '', file_backup = '', server_name= '', server_backup = '', path = '/content/drive/MyDrive/minecraft'):
    # Get the default server
    server_name = SERVER_IN_USE(server_name)
    # Settings path
    if file_name != '':
      path1 = path + f'/{server_name}/' + file_name
      if server_backup != '':
        if exists(f'{drive_path}/{server_backup}') == False:
          !mkdir '{drive_path}/{server_backup}'
      if file_backup != '' and server_backup != '': path2 = path + f'/{server_backup}/' + file_backup
      elif file_backup != '' and server_backup == '': path2 = path + f'/{server_name}/' + file_backup
      elif file_backup == '' and server_backup != '':  path2 = path + f'/{server_backup}/' + '(back-up)'
      else: path2 = path + f'/{server_name}/' + '(back-up)'
    else:
      path1 = path + f'/{server_name}'
      if server_backup != '': path2 = path + f'/{server_backup}'
      else: path2 = path + f'/{server_name}' + '(back-up)'
    # Checking and zipping
    if exists(path1) == False: raise Exception(' Creating your minecraft server before backing up files')
    if exists(path2) == True: raise Exception(' Back up path exists')
    # Zipping
    !zip -r $path2.zip $path1 &>dev/null && echo "Zipping done!" || echo "Zipping faled."
    # Download
    choice = input('Download? (y/n) : ')
    if choice == 'y':
      files.download(f'{path2}.zip')
  @staticmethod
  def Map(server_name, choice, world):
    server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{server_name}/server.properties") == False: raise Exception(' Running your minecraft server before editing properties')
    else:
      LOG("This only applied on .zip files . Others won't be supported")
      if choice == 'url':
        # Download file
        url = input('Download link : ')
        DOWNLOAD_FILE(url= url, path= f"{drive_path}/{server_name}", file_name= 'tmp.zip')
        # Unzipping
        !unzip tmp.zip -d $world >/dev/null
      else:
        %cd $drive_path/$server_name
        # Download file
        uploaded = files.upload()
        files = [fn for fn in uploaded.keys()][0] # Get the name of the uploaded file
        # Unzipping
        !unzip file_name -d $world >/dev/null
        %cd $drive_path
#------------------------------------------------------------------------------------------------------------------------------------#
!sudo apt update &>/dev/null &&echo 'apt cache successfully updated' || echo "apt cache update failed, you might receive stale packages"
!sudo apt upgrade &>/dev/null &&echo 'apt cache successfully upgraded' || echo "apt cache upgrade failed, you might receive stale packages"
if exists(drive_path) == False:
  !mkdir -p $drive_path &>/dev/null&
if exists(SERVERCONFIG) == False:
  dump({"server_list": [], "server_in_use": "", "ngrok_proxy" : {'authtoken' : '', "region" : ''}}, open(SERVERCONFIG, 'w'))
%cd $drive_path
LOG('Completed')

In [ ]:
# @markdown ####**Eligiendo servidor de Minecraft**
# @markdown ##### Ayudando a elegir el nombre por defecto de server_name.
Server.Choose_server()

---

# **Inciando**
---


In [ ]:
# @markdown ####**Elige el nombre del archivo del servidor**
server_name = "Minecraft-server" # @param {type:"string"}

Server(

  #------------------------------#
    server_name = server_name,
    server_type = "vanilla", # default server type -- Please don't change this !!
    # Change to emty strings to get the full list of avalable version
    # Remember to set the versions variable in install server function to 'vanilla - latest_version'
    #if you wanna to install the latest server.jar
    version = 'vanilla - latest_version', #

).Install_server(

    # @markdown ####Puedes usar [ngrok](https://ngrok.com/), [playit.gg](https://playit.gg/) o [Cloudflare's Argo](https://www.cloudflare.com/). Ngrok es fácil de configurar y no requiere que los clientes instalen nada, pero a menudo puede ser poco confiable. Argo no tiene muchas limitaciones pero requiere un poco más de trabajo. Playit proporciona versiones limitadas pero con un dominio estático.
    # Servicio de turnel estándar (turnel disponible: ngrok, argo, playit)
    tunnel_service = 'ngrok' #@param ['ngrok', 'argo', 'playit']

)

In [ ]:
# @markdown ####**Eliminar Servidor**
Server(server_name = '').Delete_server()

-----------------------------------------------------

# **Iniciar o Parar la consola**
---
La consola para tu servidor de minecraft

 - **[NGROK](https://ngrok.com):**
  - La IP cambiará siempre que reinicies el servidor.
  - Tienes que poner el TOKEN de Ngrok abajo, entra en este link https://dashboard.ngrok.com/auth para obtener tu token (tienes que tener una cuenta creada en ngrok)
  - Puedes cambiar en este codigo más abajo la REGIÓN, esto es importante para el LAG y PING. Usa las regiones más cercanas a ti. La región predeterminada es Estados Unidos, la cuál no es recomendada para paises de America del sur o España. Para estos se pueden usar regiones como 'sa' (Brasil) y 'eu' (Europa). Puedes cambiarlo en este código de abajo donde esta señalado.

- **[Cloudflare's argo](https://www.cloudflare.com/)** :
    - Si aparece el mensaje '¡Tu túnel gratuito ha comenzado!' aparecerá la notificación => Listo.
    - Acceso a tu servidor:
    1. Descargua [Cloudflared client](https://github.com/cloudflare/cloudflared/releases/).
    2. Inicie el binario con `<tu nombre de archivo de Cloudflare> accede tcp --hostname <dirección_túnel> --url 127.0.0.1:25565` (nota: dirección_túnel es su dirección que se ha configurado en su Cloudflare)
    4. Finalmente, conéctese a `127.0.0.1:25565` desde el cliente de Minecraft que se encuentra en esa máquina.
  
- **[PLAYIT](https://playit.gg/):**
  - Tienes que darle click al link que aparecerá, ahí tendrás que crear un tunnel para Minecraft y usar la IP que te dará la aplicación.

In [ ]:
Cmd(
    #------------------------------#
    # Leaving emty string   -> set to default files
    # @markdown
    server_name = ''
    #------------------------------#
).Run_server()

---

# **Software**
---
Cambia el software (vanilla) por otros

---


+ Verifica si su servidor de Minecraft está en Gdrive o no.


In [ ]:
Ulities.Software(
  # Default server
  server_name = '',

  # Server version
  version = '',

  # @markdown ####**Elegir tipo de servidor**
  server_type = "vanilla" # @param ["vanilla", "snapshot", "paper", "purpur", "forge", "fabric", 'velocity', 'arclight','mohist', 'banner']

)

---